In [13]:
# Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import ray
from ray import tune
import seqdatasets
import torch
from eugene import preprocess as pp
from eugene.train._hyperopt import hyperopt
from eugene.models import get_model
import importlib
from eugene.models import SequenceModule
from eugene.models.zoo import CNN
from eugene import models

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
ray.init(num_cpus=4, num_gpus=1)

2023-05-25 14:27:09,082	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.4.0


In [3]:
sdata = seqdatasets.random1000()
pp.ohe_seqs_sdata(sdata)
pp.make_unique_ids_sdata(sdata)
pp.train_test_split_sdata(sdata)
sdata["ohe_seq"] = sdata["ohe_seq"].transpose("_sequence", "_ohe", "length")
sdata

0it [00:00, ?it/s]

1000it [00:00, 2402.32it/s]


<xarray.Dataset>
Dimensions:     (_sequence: 1000, _ohe: 4, length: 100, id: 1000)
Coordinates:
  * id          (id) <U7 'seq0000' 'seq0001' 'seq0002' ... 'seq0998' 'seq0999'
Dimensions without coordinates: _sequence, _ohe, length
Data variables: (12/24)
    activity_0  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_1  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_2  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_3  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_4  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_5  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ...          ...
    label_8     (_sequence) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
    label_9     (_sequence) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
    name        (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>
    seq         (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>
    ohe_seq     (_sequence, _ohe, length) uint8 0 0 0 0 1 0 0 ... 1 0 0 0 0 0 0
    train_val   (id) bool True False False False ... True False False False
Attributes:
    max_jitter:  0

In [4]:
tune_config = {
    "module": "SequenceModule",
    "model": {
    "arch_name": "CNN",
    "model_name": "hyper_funkytiger",
    "arch": {
      "input_len": 100,
      "output_dim": 2,
      "conv_kwargs": {
        "input_channels": 4,
        "conv_channels": [tune.choice([32, 64, 128, 256, 512]), tune.choice([32, 64, 128, 256, 512]), tune.choice([32, 64, 128, 256, 512])],
        "conv_kernels": [tune.choice([3, 5, 7]), tune.choice([3, 5, 7]), tune.choice([3, 5, 7])],
        "pool_kernels": [tune.choice([1, 2]), tune.choice([1, 2]), tune.choice([1, 2])],
        "dropout_rates": 0.1,
        "batchnorm": True
      },
      "dense_kwargs": {
        "hidden_dims": [tune.choice([32, 64, 128, 256, 512]), tune.choice([32, 64, 128, 256, 512])],
      },
    },
    "task": "regression",
    "loss_fxn": "mse",
    "optimizer": "adam",
    "optimizer_lr": tune.loguniform(1e-4, 1e-1),
  },
  "batch_size": tune.choice([32, 64, 128]),
}

In [5]:
res = hyperopt(
    tune_config,
    sdata=sdata,
    target_keys=["activity_0", "activity_1"],
    train_key="train_val",
    epochs=5,
    seq_transforms={"ohe_seq": lambda x: torch.tensor(x, dtype=torch.float32)},
    gpus=1,
    cpus=4,
    num_samples=5,
    num_workers=0
)

Performing hyperparameter optimization using HyperOpt.


/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(hyperopt_with_tune pid=3565229) {'module': 'SequenceModule', 'model': {'arch_name': 'CNN', 'model_name': 'hyper_funkytiger', 'arch': {'input_len': 100, 'output_dim': 2, 'conv_kwargs': {'input_channels': 4, 'conv_channels': [512, 128, 64], 'conv_kernels': [5, 7, 7], 'pool_kernels': [2, 1, 2], 'dropout_rates': 0.1, 'batchnorm': True}, 'dense_kwargs': {'hidden_dims': [512, 32]}}, 'task': 'regression', 'loss_fxn': 'mse', 'optimizer': 'adam', 'optimizer_lr': 0.0033596391919626938}, 'batch_size': 32}
(hyperopt_with_tune pid=3565229) Model: SequenceModule(
(hyperopt_with_tune pid=3565229)   (arch): CNN(
(hyperopt_with_tune pid=3565229)     (conv1d_tower): Conv1DTower(
(hyperopt_with_tune pid=3565229)       (layers): Sequential(
(hyperopt_with_tune pid=3565229)         (0): Conv1d(4, 512, kernel_size=(5,), stride=(1,), padding=valid)
(hyperopt_with_tune pid=3565229)         (1): ReLU()
(hyperopt_with_tune pid=3565229)         (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil

(hyperopt_with_tune pid=3565229) [rank: 0] Global seed set to 13


(hyperopt_with_tune pid=3565229) Dropping 0 sequences with NaN targets.


...

(hyperopt_with_tune pid=3565806) /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(hyperopt_with_tune pid=3565806)   rank_zero_warn(


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s]                             


(hyperopt_with_tune pid=3565806) /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(hyperopt_with_tune pid=3565806)   rank_zero_warn(
(hyperopt_with_tune pid=3565806) /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
(hyperopt_with_tune pid=3565806)   rank_zero_warn(


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.35it/s, v_num=6]
Validation: 0it [00:00, ?it/s]3565806) 
(hyperopt_with_tune pid=3565806) 
Epoch 1: 100%|██████████| 1/1 [00:00<00:00, 24.69it/s, v_num=6]
Validation: 0it [00:00, ?it/s]3565806) 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(hyperopt_with_tune pid=3565806) 
Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 35.78it/s, v_num=6]
Validation: 0it [00:00, ?it/s]3565806) 
(hyperopt_with_tune pid=3565806) 
Epoch 3: 100%|██████████| 1/1 [00:00<00:00, 24.73it/s, v_num=6]
Validation: 0it [00:00, ?it/s]3565806) 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(hyperopt_with_tune pid=3565806) 
Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 30.80it/s, v_num=6]
Validation: 0it [00:00, ?it/s]3565806) 
Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  1.81it/s, v_num=6]


(hyperopt_with_tune pid=3565806) `Trainer.fit` stopped: `max_epochs=5` reached.
2023-05-25 14:29:53,150	INFO tune.py:945 -- Total run time: 158.24 seconds (158.11 seconds for the tuning loop).


In [13]:
res.results

{'eb1a0_00000': {'loss': 0.12786975502967834,
  'time_this_iter_s': 0.3564581871032715,
  'done': True,
  'training_iteration': 5,
  'trial_id': 'eb1a0_00000',
  'date': '2023-05-25_14-27-50',
  'timestamp': 1685050070,
  'time_total_s': 8.50845456123352,
  'pid': 3565229,
  'hostname': 'carter-gpu-01',
  'node_ip': '192.168.169.178',
  'config': {'model': {'arch_name': 'CNN',
    'model_name': 'hyper_funkytiger',
    'task': 'regression',
    'loss_fxn': 'mse',
    'optimizer': 'adam',
    'optimizer_lr': 0.0033596391919626938},
   'batch_size': 32},
  'time_since_restore': 8.50845456123352,
  'iterations_since_restore': 5,
  'experiment_tag': '0_batch_size=32,0=512,1=128,2=64,0=5,1=7,2=7,0=2,1=1,2=2,0=512,1=32,optimizer_lr=0.0034'},
 'eb1a0_00001': {'loss': 0.32374894618988037,
  'time_this_iter_s': 0.0982823371887207,
  'done': True,
  'training_iteration': 5,
  'trial_id': 'eb1a0_00001',
  'date': '2023-05-25_14-28-19',
  'timestamp': 1685050099,
  'time_total_s': 6.802790880203247

In [22]:
from os import PathLike
from typing import List, Union
import seqdata as sd
import numpy as np
import xarray as xr
import ray
import importlib
from eugene import dataload as dl
from eugene import models, settings
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from ray import tune
from ray.air import session
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.schedulers import (ASHAScheduler, MedianStoppingRule,
                                 PopulationBasedTraining)
from ray.tune.search import BasicVariantGenerator
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.hyperopt import HyperOptSearch
from torch.utils.data import DataLoader

from eugene.train._hyperopt import hyperopt_with_tune
scheduler_dict = {
    "ASHAScheduler": ASHAScheduler,
    "MedianStoppingRule": MedianStoppingRule,
    "PopulationBasedTraining": PopulationBasedTraining,
    
}
default_scheduler_args = {
    "ASHAScheduler": {"max_t":10, "grace_period":1, "reduction_factor":2},
    "MedianStoppingRule": {"grace_period":1},
    "PopulationBasedTraining": {"grace_period":1},
}
algo_dict = {
    "BayesOptSearch": BayesOptSearch,
    "HyperOptSearch": HyperOptSearch,
    "BasicVariantGenerator": BasicVariantGenerator,
}
default_algo_args = {
    "BayesOptSearch": {"metric":"loss", "mode":"min"},
    "HyperOptSearch": {},
    "BasicVariantGenerator": {},
}

def hyperopt(
    config,
    sdata = None,
    seq_key: str = "ohe_seq",
    target_keys: Union[str, List[str]] = None,
    train_key: str = "train_val",
    scheduler="ASHAScheduler",
    algorithm="BasicVariantGenerator",
    num_samples: int = 10,
    epochs: int = 10,
    gpus: int = None,
    cpus: int = 1,
    num_workers: int = None,
    log_dir: PathLike = None,
    name: str = None,
    version: str = None,
    train_dataloader: DataLoader = None,
    val_dataloader: DataLoader = None,
    seq_transforms: List[str] = None,
    seed: int = None,
    scheduler_kwargs: dict = None,
    algorithm_kwargs: dict = None,
    **kwargs
):
    """Perform hyperparameter optimization using HyperOpt."""
    trainable = tune.with_parameters(
        hyperopt_with_tune,
        sdata=sdata,
        seq_key=seq_key,
        target_keys=target_keys,
        train_key=train_key,
        epochs=epochs,
        gpus=gpus,
        num_workers=num_workers,
        log_dir=log_dir,
        name=name,
        version=version,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        seq_transforms=seq_transforms,
        seed=seed,
        **kwargs
    )
    if scheduler_kwargs is None or len(scheduler_kwargs) == 0:
        scheduler_kwargs = default_scheduler_args[scheduler]
    scheduler = scheduler_dict[scheduler](**scheduler_kwargs)
    if algorithm_kwargs is None or len(algorithm_kwargs) == 0:
        algorithm_kwargs = default_algo_args[algorithm]
    algo = algo_dict[algorithm](**algorithm_kwargs)
    print(f"Using {scheduler} and {algo}")
    analysis = tune.run(
        trainable,
        config=config,
        scheduler=scheduler,
        search_alg=algo,
        metric="loss",
        mode="min",
        num_samples=num_samples,
        local_dir=settings.logging_dir,
        keep_checkpoints_num=1,
        checkpoint_score_attr="min-val_loss_epoch",
        resources_per_trial={
            "cpu": cpus,
            "gpu": gpus
        },
        name=name
    )
    return analysis

In [25]:
res = hyperopt(
    tune_config,
    sdata=sdata,
    target_keys=["activity_0", "activity_1"],
    train_key="train_val",
    epochs=5,
    seq_transforms={"ohe_seq": lambda x: torch.tensor(x, dtype=torch.float32)},
    gpus=1,
    cpus=4,
    num_samples=5,
    num_workers=0,
    algorithm="HyperOptSearch",
)

AssertionError: HyperOpt must be installed! Run `pip install hyperopt`.

In [21]:
res.results_df

,loss,time_this_iter_s,done,training_iteration,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore,experiment_tag,config/batch_size,config/model/arch_name,config/model/model_name,config/model/task,config/model/loss_fxn,config/model/optimizer,config/model/optimizer_lr
trial_id,,,,,,,,,,,,,,,,,,,,
8049e_00000,0.387606,0.196039,True,5,2023-05-25_14-39-22,1685050762,8.916179,3566673,carter-gpu-01,192.168.169.178,8.916179,5,"0_batch_size=64,0=64,1=32,2=32,0=3,1=5,2=5,0=1...",64,CNN,hyper_funkytiger,regression,mse,adam,0.004630
8049e_00001,0.648861,9.289732,True,1,2023-05-25_14-40-12,1685050812,9.289732,3566793,carter-gpu-01,192.168.169.178,9.289732,1,"1_batch_size=32,0=512,1=128,2=256,0=7,1=3,2=7,...",32,CNN,hyper_funkytiger,regression,mse,adam,0.000289
8049e_00002,0.798076,8.374782,True,1,2023-05-25_14-40-59,1685050859,8.374782,3567012,carter-gpu-01,192.168.169.178,8.374782,1,"2_batch_size=128,0=512,1=256,2=64,0=7,1=7,2=3,...",128,CNN,hyper_funkytiger,regression,mse,adam,0.000705
8049e_00003,1452.268311,7.475911,True,1,2023-05-25_14-41-39,1685050899,7.475911,3567120,carter-gpu-01,192.168.169.178,7.475911,1,"3_batch_size=128,0=128,1=64,2=512,0=5,1=3,2=3,...",128,CNN,hyper_funkytiger,regression,mse,adam,0.023478
8049e_00004,9.155476,8.924828,True,1,2023-05-25_14-42-24,1685050944,8.924828,3567220,carter-gpu-01,192.168.169.178,8.924828,1,"4_batch_size=64,0=32,1=128,2=256,0=7,1=7,2=5,0...",64,CNN,hyper_funkytiger,regression,mse,adam,0.001059


In [ ]:
res.results_df.to_csv("hyperopt_results.csv")

# DONE!

---

# Scratch

## Loading a model from a hyperparm config

In [ ]:
config = {
    "module": "SequenceModule",
    "model": {
      "arch_name": "CNN",
      "model_name": "hyper_funkytiger",
      "arch": {
        "input_len": 100,
        "output_dim": 1,
        "conv_kwargs": {
          "input_channels": 4,
          "conv_channels": [256, 256, 256],
          "conv_kernels": [3, 3, 3],
          "pool_kernels": [1, 1, 1],
          "dropout_rates": 0.1,
          "batchnorm": True
        },
        "dense_kwargs": {
          "hidden_dims": [256, 256],
        },
      },
      "task": "regression",
      "loss_fxn": "mse",
      "optimizer": "adam",
      "optimizer_lr": 1e-3,
      "batch_size": 128
    }
}

In [ ]:
module_name = config.pop("module")
batch_size = config["model"].pop("batch_size")
arch_config = config["model"].pop("arch")
arch_name = config["model"]["arch_name"]
arch_type = getattr(importlib.import_module("eugene.models.zoo"), arch_name)
arch = arch_type(**arch_config)
module_type = getattr(importlib.import_module("eugene.models"), module_name)
model = module_type(arch=arch, **config["model"])
print(f"Model: {model}")
models.init_weights(model)

Model: SequenceModule(
  (arch): CNN(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 256, kernel_size=(3,), stride=(1,), padding=valid)
        (1): ReLU()
        (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout(p=0.1, inplace=False)
        (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=valid)
        (6): ReLU()
        (7): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
        (8): Dropout(p=0.1, inplace=False)
        (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=valid)
        (11): ReLU()
        (12): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
        (13): Dropout(p=0.1, inplace=False)
        (14): BatchNorm1d(256,